In [16]:
import re
import time
import requests
import pandas as ps
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from datetime import datetime # add this in the code and make a column that will store the date and time of the data scraping 

url = 'https://modxcomputers.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
pcComponents_url_list = []
for hrefs in soup.find_all('a', href=True):
    # print(hrefs['href'])
    if "https://modxcomputers.com/product-category/pc-components/" in hrefs['href']:
        url = urljoin(url, hrefs['href'])
        if len(urlparse(url).path.split('/')) <= 5:
            pcComponents_url_list.append(hrefs['href'])

pcComponents_url_lists = list(dict.fromkeys(pcComponents_url_list))

product_url_list = []
product_name_list = []
product_rating_list = []
product_price_list = []
product_in_stock_list = []
product_scraped_date_time = []

for link in pcComponents_url_lists:
    try:
        time.sleep(0.5)
        product_page = requests.get(link, headers=headers)
        product_soup = BeautifulSoup(product_page.content, 'html.parser')
        # this is to get the product details for products with multiple pages.
        if product_soup.find('div',{"class":"wd-loop-footer products-footer"}):
            pages = product_soup.find('div',{"class":"wd-loop-footer products-footer"}).find('ul',{"class":"page-numbers"}).find_all('li')[-2].text
            print(f"the link is : {link}, and no of pages are : {pages}")
            for page in range(1, int(pages)+1):
                time.sleep(1)
                inner_page_url = f"{link}page/{page}/"
                # print(inner_page_url)
                inner_page = requests.get(inner_page_url, headers=headers)
                inner_soup = BeautifulSoup(inner_page.content, 'html.parser')
                inner_products_blocks = inner_soup.find_all('div',{"class":"product-wrapper"})
                for product in inner_products_blocks:

                    product_url = product.find('div',{"class":"product-element-bottom"}).find('h3').find('a')['href']
                    if product_url:
                        product_url_list.append(product_url)
                    else:
                        product_url_list.append('Not Found')
                    
                    product_name = product.find('div',{"class":"product-element-bottom"}).find('h3').text
                    if product_name:
                        product_name_list.append(product_name)
                    else:
                        product_name_list.append('Not Found')
                    
                    product_rating = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wd-star-rating"}).text.strip().split(' ')[1]
                    if product_rating:
                        product_rating_list.append(product_rating)
                    else:
                        product_rating_list.append('Not Found')
                    
                    product_price = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wrap-price"}).text.strip()
                    if product_price:
                        product_price_list.append(product_price)
                    else:
                        product_price_list.append('Not Found')
                    
                    product_in_stock = product.find('div',{"class":"product-element-bottom"}).find('p').text.strip()
                    if product_in_stock:
                        product_in_stock_list.append(product_in_stock)
                    else:
                        product_in_stock_list.append('Not Found')
                    
                    now_time = datetime.now().strftime('%d/%m/%Y-%H:%M')
                    product_scraped_date_time.append(now_time)

        # this is to get the product details for products with one page.
        else:
            time.sleep(1)
            print(f"the link is : {link}, and no of pages are : 0")
            inner_page = requests.get(link, headers=headers)
            inner_soup = BeautifulSoup(inner_page.content, 'html.parser')
            inner_products_blocks = inner_soup.find_all('div',{"class":"product-wrapper"})

            for product in inner_products_blocks:
                
                product_url = product.find('div',{"class":"product-element-bottom"}).find('h3').find('a')['href']
                if product_url:
                    product_url_list.append(product_url)
                else:
                    product_url_list.append('Not Found')
                
                product_name = product.find('div',{"class":"product-element-bottom"}).find('h3').text
                if product_name:
                    product_name_list.append(product_name)
                else:
                    product_name_list.append('Not Found')
                
                product_rating = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wd-star-rating"}).text.strip().split(' ')[1]
                if product_rating:
                    product_rating_list.append(product_rating)
                else:
                    product_rating_list.append('Not Found')
                
                product_price = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wrap-price"}).text.strip()
                if product_price:
                    product_price_list.append(product_price)
                else:
                    product_price_list.append('Not Found')
                
                product_in_stock = product.find('div',{"class":"product-element-bottom"}).find('p').text.strip()
                if product_in_stock:
                    product_in_stock_list.append(product_in_stock)
                else:
                    product_in_stock_list.append('Not Found')
                
                now_time = datetime.now().strftime('%d/%m/%Y-%H:%M')
                product_scraped_date_time.append(now_time)

    except Exception as e:
        raise(f'Exception occurred : {e}')

data = {
    'product_scraped_date_time' : product_scraped_date_time,
    'product_name' : product_name_list,
    'product_rating_of_5' : product_rating_list,
    'product_price' : product_price_list,
    'product_in_stock' : product_in_stock_list,
    'product_url' : product_url_list
}

df = pd.DataFrame(data)

the link is : https://modxcomputers.com/product-category/pc-components/processor/, and no of pages are : 5
the link is : https://modxcomputers.com/product-category/pc-components/motherboard/, and no of pages are : 13
the link is : https://modxcomputers.com/product-category/pc-components/cpu-cooler/, and no of pages are : 9
the link is : https://modxcomputers.com/product-category/pc-components/ram/, and no of pages are : 4
the link is : https://modxcomputers.com/product-category/pc-components/graphics-card/, and no of pages are : 6
the link is : https://modxcomputers.com/product-category/pc-components/ssd/, and no of pages are : 4
the link is : https://modxcomputers.com/product-category/pc-components/hard-drive/, and no of pages are : 0
the link is : https://modxcomputers.com/product-category/pc-components/power-supply/, and no of pages are : 5
the link is : https://modxcomputers.com/product-category/pc-components/cabinet/, and no of pages are : 13
the link is : https://modxcomputers.co

In [27]:
import re
import time
import requests
import pandas as pd
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from datetime import datetime

class WebScraper:
    def __init__(self, base_url, user_agent, headers=None):
        self.base_url = base_url
        self.user_agent = user_agent
        self.headers = headers
        self.pcComponents_url_list = []
        self.product_data = []
        self.product_url_list = []
        self.product_name_list = []
        self.product_rating_list = []
        self.product_price_list = []
        self.product_in_stock_list = []
        self.product_scraped_date_time = []

    def _get_soup(self, url):
        page = requests.get(url, headers=self.headers)
        return BeautifulSoup(page.content, 'html.parser')

    def scrape(self):
        soup = self._get_soup(self.base_url)
        for hrefs in soup.find_all('a', href=True):
            if "https://modxcomputers.com/product-category/pc-components/" in hrefs['href']:
                url = urljoin(base_url, hrefs['href'])
                if len(urlparse(url).path.split('/')) <= 5:
                    self.pcComponents_url_list.append(hrefs['href'])

        self.pcComponents_url_list = list(dict.fromkeys(self.pcComponents_url_list))
        
        for link in self.pcComponents_url_list:
            try:
                time.sleep(0.5)
                product_page = requests.get(link, headers=self.headers)
                product_soup = BeautifulSoup(product_page.content, 'html.parser')
                # this is to get the product details for products with multiple pages.
                if product_soup.find('div',{"class":"wd-loop-footer products-footer"}):
                    pages = product_soup.find('div',{"class":"wd-loop-footer products-footer"}).find('ul',{"class":"page-numbers"}).find_all('li')[-2].text
                    print(f"the link is : {link}, and no of pages are : {pages}")
                    for page in range(1, int(pages)+1):
                        time.sleep(1)
                        inner_page_url = f"{link}page/{page}/"
                        # print(inner_page_url)
                        inner_page = requests.get(inner_page_url, headers=self.headers)
                        inner_soup = BeautifulSoup(inner_page.content, 'html.parser')
                        inner_products_blocks = inner_soup.find_all('div',{"class":"product-wrapper"})
                        for product in inner_products_blocks:

                            product_url = product.find('div',{"class":"product-element-bottom"}).find('h3').find('a')['href']
                            if product_url:
                                self.product_url_list.append(product_url)
                            else:
                                self.product_url_list.append('Not Found')
                            
                            product_name = product.find('div',{"class":"product-element-bottom"}).find('h3').text
                            if product_name:
                                self.product_name_list.append(product_name)
                            else:
                                self.product_name_list.append('Not Found')
                            
                            product_rating = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wd-star-rating"}).text.strip().split(' ')[1]
                            if product_rating:
                                self.product_rating_list.append(product_rating)
                            else:
                                self.product_rating_list.append('Not Found')
                            
                            product_price = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wrap-price"}).text.strip()
                            if product_price:
                                self.product_price_list.append(product_price)
                            else:
                                self.product_price_list.append('Not Found')
                            
                            product_in_stock = product.find('div',{"class":"product-element-bottom"}).find('p').text.strip()
                            if product_in_stock:
                                self.product_in_stock_list.append(product_in_stock)
                            else:
                                self.product_in_stock_list.append('Not Found')
                            
                            self.product_scraped_date_time.append(datetime.now().strftime('%d/%m/%Y-%H:%M'))

                # this is to get the product details for products with one page.
                else:
                    time.sleep(1)
                    print(f"the link is : {link}, and no of pages are : 0")
                    inner_page = requests.get(link, headers=headers)
                    inner_soup = BeautifulSoup(inner_page.content, 'html.parser')
                    inner_products_blocks = inner_soup.find_all('div',{"class":"product-wrapper"})

                    for product in inner_products_blocks:
                        
                        product_url = product.find('div',{"class":"product-element-bottom"}).find('h3').find('a')['href']
                        if product_url:
                            self.product_url_list.append(product_url)
                        else:
                            self.product_url_list.append('Not Found')
                        
                        product_name = product.find('div',{"class":"product-element-bottom"}).find('h3').text
                        if product_name:
                            self.product_name_list.append(product_name)
                        else:
                            self.product_name_list.append('Not Found')
                        
                        product_rating = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wd-star-rating"}).text.strip().split(' ')[1]
                        if product_rating:
                            self.product_rating_list.append(product_rating)
                        else:
                            self.product_rating_list.append('Not Found')
                        
                        product_price = product.find('div',{"class":"product-element-bottom"}).find('div',{"class":"wrap-price"}).text.strip()
                        if product_price:
                            self.product_price_list.append(product_price)
                        else:
                            self.product_price_list.append('Not Found')
                        
                        product_in_stock = product.find('div',{"class":"product-element-bottom"}).find('p').text.strip()
                        if product_in_stock:
                            self.product_in_stock_list.append(product_in_stock)
                        else:
                            self.product_in_stock_list.append('Not Found')

                        self.product_scraped_date_time.append(datetime.now().strftime('%d/%m/%Y-%H:%M'))
            except Exception as e:
                raise(f'Exception occurred : {e}')

            self.product_data = {
                    'product_scraped_date_time' : self.product_scraped_date_time,
                    'product_name' : self.product_name_list,
                    'product_rating_of_5' : self.product_rating_list,
                    'product_price' : self.product_price_list,
                    'product_in_stock' : self.product_in_stock_list,
                    'product_url' : self.product_url_list
            }

    def to_dataframe(self):
        df = pd.DataFrame(self.product_data)
        return df

    def run(self):
        self.scrape()
        df = self.to_dataframe()
        return df

if __name__=="__main__":
    base_url = 'https://modxcomputers.com/'
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
    headers = {'User-Agent': user_agent}

    scraper = WebScraper(base_url, user_agent, headers)
    df = scraper.run()

    df.to_csv('scraped_data.csv', index=False)

the link is : https://modxcomputers.com/product-category/pc-components/processor/, and no of pages are : 5
the link is : https://modxcomputers.com/product-category/pc-components/motherboard/, and no of pages are : 13
the link is : https://modxcomputers.com/product-category/pc-components/cpu-cooler/, and no of pages are : 9
the link is : https://modxcomputers.com/product-category/pc-components/ram/, and no of pages are : 4
the link is : https://modxcomputers.com/product-category/pc-components/graphics-card/, and no of pages are : 6
the link is : https://modxcomputers.com/product-category/pc-components/ssd/, and no of pages are : 4
the link is : https://modxcomputers.com/product-category/pc-components/hard-drive/, and no of pages are : 0
the link is : https://modxcomputers.com/product-category/pc-components/power-supply/, and no of pages are : 5
the link is : https://modxcomputers.com/product-category/pc-components/cabinet/, and no of pages are : 13
the link is : https://modxcomputers.co